## [Day 19](https://adventofcode.com/2020/day19)

So this one has a series of instructions which we're to check if match strings in the 'messages' given. They look an awful lot like regular expressions where a pipe means 'or'. It also seems to have a tree like structure in that there are various nodes with possible branches from it.

Our ultimate goal is to check each message against rule zero

In [120]:
import pandas as pd
import numpy as np
import itertools, re, copy
imp = open('../inputs/d19.txt').read().splitlines()
instructions = imp[:imp.index('')]
messages = imp[imp.index('')+1:]
instructions[:5]

['50: 5 18 | 61 39',
 '79: 118 18 | 125 39',
 '102: 38 39 | 19 18',
 '114: 39 38 | 18 100',
 '119: 39 103']

In [121]:
messages[:5]

['abbbbabbbaabababaaabbaabbbbbbbba',
 'bbbabbbbabbabbbbabbbabab',
 'aabbababbbbababbaaabbabb',
 'aababbaaaabbbaaaaabaabbb',
 'aaaabaaababbbbbbbbbbabaa']

In [122]:
# Okay so I think I want to turn all these instructions into lists of tuples separated by the pipes if they exist

keys = [x.split(':')[0] for x in instructions]
values = [x.split(':')[1][1:] for x in instructions]

def split_to_tuple(x):
    ret = []
    pieces = x.split('|')
    for piece in pieces:
        piece = piece.strip().split(' ')
        piece = [x.replace('"', '') for x in piece]
        ret.append(piece)
    return ret
values = [split_to_tuple(x) for x in values]
instructions = dict(zip(keys, values))
# save a copy
instructions_old = copy.deepcopy(instructions)
instructions['50']

[['5', '18'], ['61', '39']]

In [123]:
# Really struggling with whether this will be easier to start with the zero-th instruction
# and go downwards or start with the ones that have and then move upwards....
# I think i'll try the latter even though I'm having some doubts about whether there 
# are some situations where I'll end up with multiple 'unresolved' instructins.

def flatten_list(l):
    return [x for sublist in l for x in sublist]

completed = {}
for key in instructions.keys():
    flat_list = flatten_list(instructions[key])
    if flat_list in [['a'], ['b']]:
        completed.update({key:flat_list[0]})
completed

{'18': 'a', '39': 'b'}

In [124]:
# This just to swap in the values of things we have done
def replace_value(l, val):
    l = copy.deepcopy(l)
    for i, sublist in enumerate(l):
        for j, x in enumerate(sublist):
            if l[i][j] == val:
                l[i][j] = completed[val]
    return l

replace_value(instructions['50'], '18')

[['5', 'a'], ['61', '39']]

In [125]:
# Are we done looking at this one:
def check_done(l):
    for x in flatten_list(l):
        if re.match(r"[0-9]", x):
            return False
    return True
check_done(instructions['50'])

False

In [126]:
check_done(instructions['18'])

True

In [127]:
# Smoosh down the contents in to a regex:
def regex_or(l):
    l = [''.join(sublist) for sublist in l]
    if len(l) > 1:
        return '(' + '|'.join(l) + ')'
    else:
        return l[0]
regex_or(instructions['50'])    

'(518|6139)'

In [128]:
# FINAL VERSION OF LOOP HERE:
checked = []
# And now we loop till she done:
while not check_done(instructions['0']):
    
    # pick an element from the completed dict that we haven't checked yet:
    next_key = ''
    i = 0
    while next_key == '':
        if list(completed.keys())[i] in checked:
            i += 1
        else:
            next_key = list(completed.keys())[i]
    
    # loop through the incomplete keys
    for key in instructions.keys():
        if key in completed.keys():
            continue
        # If we find a match:
        if next_key in flatten_list(instructions[key]):
            # Replace and update
            new_val = replace_value(instructions[key], next_key)
            # If it's done, add it to the list
            if check_done(new_val):
                new_val = regex_or(new_val)
                completed.update({key:new_val})
            # In any case update the instructions list
            instructions.update({key:new_val})
    # mark this off as having been done
    checked += [next_key]

In [129]:
pattern = re.compile('^' + instructions['0'] + '$')
sol1 = 0
for message in messages:
    if pattern.match(message):
        sol1 +=1
sol1

233

### Part 2
For the second part of this question we're asked to change two of the instructions and repeat the process. The catch is that there is a loop involved in the instructions now. We're also cautioned against trying to find a generalized solution to the problems but instead we should look to figure out the exact issues with this substitution. 

This is the subsitution:  
8: 42 | 42 8  
11: 42 31 | 42 11 31

In [130]:
# Ok we add this to the dictionary:
instructions_pt2 = copy.deepcopy(instructions_old)
instructions_pt2.update({'8': [['42'], ['42', '8']], '11': [['42', '31'], ['42', '11', '31']]})
# So the issues are that when we have an '8' in the expresssion, it could be like this (42)*8
# and the same with 11 ~ 42*11

# Unfortunately, my method of programming this did not actually place the individual numbers in there 

In [131]:
instructions_pt2['8']

[['42'], ['42', '8']]

In [132]:
instructions_pt2['11']

[['42', '31'], ['42', '11', '31']]

In [133]:
instructions_pt2['0']

[['8', '11']]

In [134]:
# Again make a list of the completed key value pairs
completed2 = {}
for key in instructions_pt2.keys():
    flat_list = flatten_list(instructions_pt2[key])
    if flat_list in [['a'], ['b']]:
        completed2.update({key:flat_list[0]})

In [135]:
# And.... loop
checked2 = []
# And now we loop till she done:
while not check_done(instructions_pt2['0']):
    
    # I think we'll just add in a little break here to fix the
    # two problem children of the house.
    if '42' in completed2.keys():
        completed2.update({'8':completed2['42'] + '+'})
        instructions_pt2.update({'8':completed2['42'] + '+'})
        if '31' in completed2.keys():
            break;
    
    # pick an element from the completed dict that we haven't checked yet:
    next_key = ''
    i = 0
    while next_key == '':
        if list(completed2.keys())[i] in checked2:
            i += 1
        else:
            next_key = list(completed2.keys())[i]
    
    # loop through the incomplete keys
    for key in instructions_pt2.keys():
        if key in completed2.keys():
            continue
        # If we find a match:
        if next_key in flatten_list(instructions_pt2[key]):
            # Replace and update
            new_val = replace_value(instructions_pt2[key], next_key)
            # If it's done, add it to the list
            if check_done(new_val):
                new_val = regex_or(new_val)
                completed2.update({key:new_val})
            # In any case update the instructions list
            instructions_pt2.update({key:new_val})
    # mark this off as having been done
    checked2 += [next_key]

In [138]:
instructions_pt2['0']

[['8', '11']]

In [143]:
instructions_pt2['31']

'((((a(b(baa|((a|b)b|ba)b)|a(a(bb|aa)|b(aa|ba)))|b((a(bb|a(a|b))|b(a|b)(a|b))b|(a(aa|ba)|b(bb|aa))a))a|(b(a(aba|(bb|aa)b)|b(((a|b)b|ba)a|(bb|aa)b))|a(a(ab|aa)(a|b)|b(b(bb|a(a|b))|a(ab|(a|b)a))))b)a|((a(((ba|ab)b|baa)b|((aa|b(a|b))a|(bb|a(a|b))b)a)|b(((ab|(a|b)a)b|(bb|aa)a)a|(bba|a(bb|ab))b))a|(b(a(a(aa|ba)|bab)|baba)|a((((a|b)b|ba)b|(bb|aa)a)a|(b(ab|(a|b)a)|a(bb|ba))b))b)b)a|(b(b(b((aba|b(a|b)(a|b))a|((bb|ba)b|(aa|b(a|b))a)b)|a(a(baa|(bb|ba)b)|b(aba|bba)))|a(b(b(b(ab|aa)|a(aa|ba))|a(aaa|b(aa|b(a|b))))|a(a(baa|(aa|ba)b)|b(b(a|b)(a|b)|a(bb|aa)))))|a((a(((aa|ba)a|aab)b|(aaa|(bb|ab)b)a)|b(b((a|b)b|ba)a|(bb|ab)bb))b|(a(a(aba|b(a|b)(a|b))|b(a(bb|ab)|b(bb|aa)))|b(bbab|(a(bb|a(a|b))|b(a|b)(a|b))a))a))b)'

In [148]:
# Now the issue is that we want something like this 'a{n}b{n}' but I don't know how to hold that number
# constant in a regular expression. I guess we can just keep doing it till we reach the max string length...
max_len = max([len(x) for x in messages])
new_messages = messages.copy()
for i in range(1, max_len):
    new_pattern = instructions_pt2['8'] + '(' + instructions_pt2['42'] +'{' + str(i) + '}?' + instructions_pt2['31'] +'{' + str(i) + '}?)'
    new_pattern = re.compile('^' + new_pattern + '$')
    for message in new_messages:
        if pattern2.match(message):
            new_messages.remove(message)
len(messages) - len(new_messages)

347

In [146]:
i = 1
instructions_pt2['8'] + instructions_pt2['42'] +'{' + str(i) + '}' + instructions_pt2['31'] +'{' + str(i) + '}'

'(a(a(b((a(ab|(a|b)a)b|((a|b)(a|b)b|(bb|ab)a)a)a|((bab|(bb|a(a|b))a)a|(((a|b)b|ba)b|(ab|(a|b)a)a)b)b)|a(a((bab|(bb|a(a|b))a)b|(baa|(bb|ba)b)a)|b((bb|a(a|b))bb|(a(bb|aa)|b(bb|ab))a)))|b(a(b(a(a(a|b)(a|b)|b(bb|ab))|b((ba|ab)b|baa))|a(((a|b)(a|b)b|(bb|ab)a)b|(a(bb|aa)|baa)a))|b(((aaa|(ab|(a|b)a)b)b|(bb|a(a|b))ba)b|((b(bb|ba)|aba)b|(bab|baa)a)a)))|b(((b(b((bb|ba)a|(a|b)(a|b)b)|a(b(bb|ba)|a(a|b)(a|b)))|a((baa|(aa|ba)b)a|((bb|ba)a|(bb|a(a|b))b)b))b|(a((a(bb|aa)|b(ab|(a|b)a))b|(b(ba|ab)|aba)a)|b((aaa|(ba|ab)b)b|(baa|aba)a))a)a|((a(a(baa|a(bb|ab))|b(b(bb|a(a|b))|a(ab|(a|b)a)))|b(a(b(a|b)(a|b)|a(ab|aa))|b((ab|aa)b|aba)))b|((((a|b)(a|b)a|aab)a|(a(bb|aa)|b(aa|ba))b)b|((b(ab|(a|b)a)|aaa)a|babb)a)a)b))+(a(a(b((a(ab|(a|b)a)b|((a|b)(a|b)b|(bb|ab)a)a)a|((bab|(bb|a(a|b))a)a|(((a|b)b|ba)b|(ab|(a|b)a)a)b)b)|a(a((bab|(bb|a(a|b))a)b|(baa|(bb|ba)b)a)|b((bb|a(a|b))bb|(a(bb|aa)|b(bb|ab))a)))|b(a(b(a(a(a|b)(a|b)|b(bb|ab))|b((ba|ab)b|baa))|a(((a|b)(a|b)b|(bb|ab)a)b|(a(bb|aa)|baa)a))|b(((aaa|(ab|(a|b)a)b)b|(bb|a

In [78]:

sol2 = 0
for message in messages:
    pattern2 = instructions_pt2['8'] + instructions_pt2['42'] +'{', + i + '}' instructions_pt2['31'] +'{', + i + '}'
    pattern2 = re.compile('^' + pattern2 + '$')
    if pattern2.match(message):
        sol2 +=1
sol2

347

In [98]:
for i in range(1,3):
    print(i)

1
2
